# Lesson 6: Exercises

## Load CADETSettings

This imports all the standard libraries and provides some helper functions.

In [ ]:
%run ../utils.ipynb

## Model templates

In [ ]:
def get_langmuir_template():
    n_comp = 2
    
    langmuir_model = get_cadet_template(n_units=4)

    # INLET
    langmuir_model.root.input.model.unit_000.unit_type = 'INLET'
    langmuir_model.root.input.model.unit_000.ncomp = n_comp
    langmuir_model.root.input.model.unit_000.inlet_type = 'PIECEWISE_CUBIC_POLY'

    # Mixer
    langmuir_model.root.input.model.unit_001.unit_type = 'CSTR'
    langmuir_model.root.input.model.unit_001.ncomp = n_comp
    langmuir_model.root.input.model.unit_001.init_volume =1e-6
    langmuir_model.root.input.model.unit_001.init_c = n_comp*[0.0]

    # Column
    langmuir_model.root.input.model.unit_002.unit_type = 'LUMPED_RATE_MODEL_WITH_PORES'
    langmuir_model.root.input.model.unit_002.ncomp = n_comp

    langmuir_model.root.input.model.unit_002.col_length = 0.6
    langmuir_model.root.input.model.unit_002.cross_section_area = 1.0386890710931253E-4
    langmuir_model.root.input.model.unit_002.col_porosity = 0.37
    langmuir_model.root.input.model.unit_002.par_porosity = 0.33
    langmuir_model.root.input.model.unit_002.par_radius = 4.5e-5

    langmuir_model.root.input.model.unit_002.col_dispersion = 2.0e-7
    langmuir_model.root.input.model.unit_002.film_diffusion = [1e-4, 1e-4]

    langmuir_model.root.input.model.unit_002.adsorption_model = 'MULTI_COMPONENT_LANGMUIR'
    langmuir_model.root.input.model.unit_002.adsorption.is_kinetic = False
    langmuir_model.root.input.model.unit_002.adsorption.mcl_ka = [2, 3]
    langmuir_model.root.input.model.unit_002.adsorption.mcl_kd = [1, 1]
    langmuir_model.root.input.model.unit_002.adsorption.mcl_qmax = [1, 1]

    langmuir_model.root.input.model.unit_002.init_c = n_comp*[0.0,]
    langmuir_model.root.input.model.unit_002.init_q = n_comp*[0.0,]

    ## Outlet
    langmuir_model.root.input.model.unit_003.ncomp = n_comp
    langmuir_model.root.input.model.unit_003.unit_type = 'OUTLET'

    set_discretization(langmuir_model, n_col=40, n_bound=n_comp*[1])
    
    return langmuir_model

def get_clr_template():
    clr_model = get_cadet_template(n_units=5)

    # INLET
    clr_model.root.input.model.unit_000.unit_type = 'INLET'
    clr_model.root.input.model.unit_000.ncomp = n_comp
    clr_model.root.input.model.unit_000.inlet_type = 'PIECEWISE_CUBIC_POLY'

    # Column
    clr_model.root.input.model.unit_001.unit_type = 'LUMPED_RATE_MODEL_WITHOUT_PORES'
    clr_model.root.input.model.unit_001.ncomp = n_comp

    clr_model.root.input.model.unit_001.col_length = 0.6
    clr_model.root.input.model.unit_001.cross_section_area = 1.0386890710931253E-4
    clr_model.root.input.model.unit_001.total_porosity = 0.6
    clr_model.root.input.model.unit_001.col_dispersion = 2.0e-7

    clr_model.root.input.model.unit_001.adsorption_model = 'MULTI_COMPONENT_LANGMUIR'
    clr_model.root.input.model.unit_001.adsorption.is_kinetic = False
    clr_model.root.input.model.unit_001.adsorption.mcl_ka = [1, 1.1]
    clr_model.root.input.model.unit_001.adsorption.mcl_kd = [1, 1]
    clr_model.root.input.model.unit_001.adsorption.mcl_qmax = [1, 1]

    clr_model.root.input.model.unit_001.init_c = n_comp*[0.0,]
    clr_model.root.input.model.unit_001.init_q = n_comp*[0.0,]

    ## Outlet
    clr_model.root.input.model.unit_002.ncomp = n_comp
    clr_model.root.input.model.unit_002.unit_type = 'OUTLET'

    set_discretization(clr_model, n_col=40, n_bound=n_comp*[1])
    
    return clr_model

## Exercise 1: Modelling dispersion of valves and tubing

Take the example from the lesson and add tubing using a `LUMPED_RATE_MODEL_WITHOUT_PORES` with $L_c = 0.5~m$, $A_c = 1 \cdot 10^{-5}~m^2$, and $D_{ax} = 1 \cdot 10^{-5}~m^2 \cdot s^{-1}$.

***Task:*** Plot the inlet and outlet of every unit operation and compare the results to a system without any considerations for valving and tubing.

***Hint:*** We can define unit operations in any order, so you can simply add to the template and then set the connections accordingly.
Make sure to also the discretization for the tubing and to set the flags for saving the solution.

## Example 2: Closed loop recycling

For difficult separations, a longer column length can be simulated by recycling the same sample several times through a column.
In `CADET`, we can model this process (called closed loop recycling (CLR)), by connecting a column unit operation with itself.

***Task:*** Use the `clr_model` template and model the CLR process.
First inject some amount on the column.
Then connect the column with itself and consider 5 recycles, each taking $3600~s$. 
Then wash the column for another cycle.